## Using *checkpoint_schedules* package

The *checkpoint_schedules* package offers a squedule of actions that enable the coordination of an adjoint-based gradient executions through a checkpoint strategy. The checkpoint schedule is built by the sequence of actions referred to as *Forward, EndForward, Reverse, Copy, EndReverse*. The actions provides functionalities such as storing the forward checkpoint data used to restart the forward solver, storing the forward checkpoint data for adjoint computations, and retrieving the stored data for both the forward solver restart and the adjoint computation. 

In the following code, we have implemented the `CheckpointingManager` class, which allows the manegement of the forward and adjoint executions in time. On using `CheckpointingManager.execute` method, we iterate over a sequence of actions given by the schedule `cp_schedule`. The actions are defined by using single-dispatch functions, where the `action` function is the generic function using the singledispatch decorator. Specific functions for different types of *checkpoint_schedules* actions are provided by using the register method of the base function `action`.

In [15]:
from checkpoint_schedules import Forward, EndForward, Reverse, Copy, EndReverse
import functools

class CheckpointingManager():
    """Manage the forward and adjoint solvers.

    Attributes
    ----------
    save_ram : int
        Number of checkpoint that will be stored in RAM.
    save_disk : int
        Number of checkpoint that will be stored on disk.
    list_actions : list
        Store the list of actions.
    max_n : int
        Total steps used to execute the solvers.
    """
    def __init__(self, max_n, save_ram, save_disk=0):
        self.max_n = max_n
        self.save_ram = save_ram
        self.save_disk = save_disk
        self.list_actions = []
        
    def execute(self, cp_schedule):
        """Execute forward and adjoint with a checkpointing strategy.

        Parameters
        ----------
        cp_schedule : object
            Checkpointing schedule.
        """
        @functools.singledispatch
        def action(cp_action):
            raise TypeError("Unexpected action")

        @action.register(Forward)
        def action_forward(cp_action):
            nonlocal model_n
            if cp_action.write_ics:
                print(("*").rjust(cp_action.n0*4))

            print(("|" + "--->"*(cp_action.n1-cp_action.n0)).rjust(cp_action.n1*4) +
                   "   "*(self.max_n - cp_action.n1 + 4) + 
                   self.list_actions[len(self.list_actions) - 1])

            n1 = min(cp_action.n1, self.max_n)
            model_n = n1
            if cp_action.n1 == self.max_n:
                cp_schedule.finalize(n1)

        @action.register(Reverse)
        def action_reverse(cp_action):
            nonlocal model_r
            print(("<---"*(cp_action.n1-cp_action.n0) + "|").rjust(cp_action.n1*4) 
                  + "   "*(self.max_n - cp_action.n1 + 4) + 
                    self.list_actions[len(self.list_actions) - 1])

            model_r += cp_action.n1 - cp_action.n0
            
        @action.register(Copy)
        def action_copy(cp_action):
            print(("+").rjust(cp_action.n*4) 
                  + "   "*(self.max_n + 4) + 
                    self.list_actions[len(self.list_actions) - 1])

    
        @action.register(EndForward)
        def action_end_forward(cp_action):
            assert model_n == self.max_n
            print("End Forward")
            
        @action.register(EndReverse)
        def action_end_reverse(cp_action):
            nonlocal model_r
            assert model_r == self.max_n
            print("End Reverse")

        model_n = 0
        model_r = 0

        count = 0
        while True:
            cp_action = next(cp_schedule)
            self.list_actions.append(str(cp_action))
            action(cp_action)
            count += 1
            if isinstance(cp_action, EndReverse):  
                break

Firstly, let us define the total steps used in the computations and set the number of time steps to store the forward checkpoint data in RAM.

In [16]:
max_n = 4 # Total number of time steps.
save_ram = 2 # Number of steps to save i RAM.

Next, let us set the `CheckpointingManager` manager object with the attributes defined above. 

In [17]:
chk_manager = CheckpointingManager(max_n, save_ram) # manager object

The *checkpoint_schedules* package provide a range o iterators for the the following checkpoint strategies:
* Revolve strategy
* Multistage 
* two-level mixed periodic/binomial
* H-Revolve
* Mixing Storage
* Periodic disk storage

We start showing how the *checkpoint_schedules* works by building a schedule iterator (named `chk_iterator`) with the Revolve strategy. The schedule is building on executing `revolver.sequence()`.

In [16]:
from checkpoint_schedules import Revolve
chk_iterator = Revolve(max_n, save_ram)
revolver.sequence()

`chk_manager.execute(revolver)` provides the executions of the forward and adjoint solvers with.

In [ ]:
chk_manager.execute(revolver)

The output above illustrates how it works the forward and adjoint executions in time with the *checkpoint_schedules* package. The symbol `|` indicates the step that the solver initialises. The symbom `*` on top of `|` indicates that the data used to restart the forward solver is stored. Whereas the symbol `+` indicates the action of copying the storage data and be used as initial condition for the forward solver recomputations. 

To complement the illustration above, consider some of the actions explained as follow:

* Forward action
    - General form: *Forward(n0, n1, write_ics, write_adj_deps, 'storage')*
    - Particular form: 
        * *Forward(0, 2, True, False, 'RAM')*:
            - Execute the forward solver from step 0 to step 2.
            - Write the forward data (*write_ics* is True) of step 0 to RAM (storage).
            - The forward data is not stored for the adjoint computation (*write_adj_deps* is False).

        * *Forward(3, 4, False, True, 'RAM')*:
            - Execute the forward solver from step 3 to step 4.
            - Do not write the forward data (*write_ics* is False) of step 4.
            - Store the forward data for the adjoint computation (*write_adj_deps* is *True*) in RAM (storage).

* Reverse action
     - General form: *Reverse(n1, n0, clear_adj_deps)*
     - Particular form:
        * *Reverse(4, 3, True)*: 
            - Execute the adjoint solver from step 4 to step 3.
            - Clear the adjoint dependencies (*clear_adj_deps* is True) used in the adjoint computation.
* Copy action
     - General form: Copy(n, from_storage, delete)
     - Particular form:
        * Copy(2, 'RAM', True):
            - Copy the forward data related to step 2 from RAM.
            - Delete the copied data from RAM (*delete* is *True*) as it is not needed anymore to restart the forward solver.

        * Copy(0, 'DISK', False):
            - Copy the forward data related to step 0 from DISK.
            - Do not delete the copied data from DISK (*delete* is *FALSE*).



In [17]:
from checkpoint_schedules import HRevolve
s = 0
revolver = HRevolve(max_n, save_ram, snap_on_disk=save_disk)
revolver.sequence()
chk_manager.execute(revolver)

*
|--->--->--->               Forward(0, 3, True, False, 'RAM')
           |--->            Forward(3, 4, False, True, 'RAM')
End Forward
           <---|            Reverse(4, 3, True)
+                        Copy(0, 'RAM', False)
|--->--->                  Forward(0, 2, False, False, 'NONE')
       |--->               Forward(2, 3, False, True, 'RAM')
       <---|               Reverse(3, 2, True)
+                        Copy(0, 'RAM', False)
|--->                     Forward(0, 1, False, False, 'NONE')
   |--->                  Forward(1, 2, False, True, 'RAM')
   <---|                  Reverse(2, 1, True)
+                        Copy(0, 'RAM', True)
|--->                     Forward(0, 1, False, True, 'RAM')
<---|                     Reverse(1, 0, True)
End Reverse


Below we have the schedule obtained from Disk-Revolve and Periodic-Disk-Revolve checkpoint strategies.

In [18]:
from checkpoint_schedules import DiskRevolve
revolver = DiskRevolve(max_n, save_ram, snap_on_disk=save_disk)
print(revolver._schedule)
revolver.sequence()
chk_manager.execute(revolver)

None
*
|--->--->--->               Forward(0, 3, True, False, 'RAM')
           |--->            Forward(3, 4, False, True, 'RAM')
End Forward
           <---|            Reverse(4, 3, True)
+                        Copy(0, 'RAM', False)
|--->--->                  Forward(0, 2, False, False, 'NONE')
       |--->               Forward(2, 3, False, True, 'RAM')
       <---|               Reverse(3, 2, True)
+                        Copy(0, 'RAM', False)
|--->                     Forward(0, 1, False, False, 'NONE')
   |--->                  Forward(1, 2, False, True, 'RAM')
   <---|                  Reverse(2, 1, True)
+                        Copy(0, 'RAM', True)
|--->                     Forward(0, 1, False, True, 'RAM')
<---|                     Reverse(1, 0, True)
End Reverse


In [19]:
from checkpoint_schedules import PeriodicDiskRevolve
revolver = PeriodicDiskRevolve(max_n, save_ram, snap_on_disk=save_disk)
revolver.sequence(period=2)
chk_manager.execute(revolver)

We use periods of size  2
*
|--->--->                  Forward(0, 2, True, False, 'disk')
       *
       |--->               Forward(2, 3, True, False, 'RAM')
           |--->            Forward(3, 4, False, True, 'RAM')
End Forward
           <---|            Reverse(4, 3, True)
       +                        Copy(2, 'RAM', True)
       |--->               Forward(2, 3, False, True, 'RAM')
       <---|               Reverse(3, 2, True)
+                        Copy(0, 'disk', False)
*
|--->                     Forward(0, 1, True, False, 'RAM')
   |--->                  Forward(1, 2, False, True, 'RAM')
   <---|                  Reverse(2, 1, True)
+                        Copy(0, 'RAM', True)
|--->                     Forward(0, 1, False, True, 'RAM')
<---|                     Reverse(1, 0, True)
End Reverse


In [20]:
from checkpoint_schedules import MixedCheckpointSchedule
snapshots = 2
revolver = MixedCheckpointSchedule(max_n, snapshots)
chk_manager.execute(revolver)

|--->                     Forward(0, 1, False, True, 'disk')
   *
   |--->--->               Forward(1, 3, True, False, 'disk')
           |--->            Forward(3, 4, False, True, 'RAM')
End Forward
           <---|            Reverse(4, 3, True)
   +                        Copy(1, 'disk', True)
   |--->                  Forward(1, 2, False, True, 'disk')
       |--->               Forward(2, 3, False, True, 'RAM')
       <---|               Reverse(3, 2, True)
   +                        Copy(1, 'disk', True)
   <---|                  Reverse(2, 1, True)
+                        Copy(0, 'disk', True)
<---|                     Reverse(1, 0, True)
End Reverse


In [25]:
from checkpoint_schedules import MultistageCheckpointSchedule
snapshots = 2
revolver = MultistageCheckpointSchedule(max_n, 0, 2)
chk_manager.execute(revolver)

*
|--->--->                  Forward(0, 2, True, False, 'disk')
       *
       |--->               Forward(2, 3, True, False, 'disk')
           |--->            Forward(3, 4, False, True, 'RAM')
End Forward
           <---|            Reverse(4, 3, True)
       +                        Copy(2, 'disk', True)
       |--->               Forward(2, 3, False, True, 'RAM')
       <---|               Reverse(3, 2, True)
+                        Copy(0, 'disk', False)
|--->                     Forward(0, 1, False, False, 'NONE')
   |--->                  Forward(1, 2, False, True, 'RAM')
   <---|                  Reverse(2, 1, True)
+                        Copy(0, 'disk', True)
|--->                     Forward(0, 1, False, True, 'RAM')
<---|                     Reverse(1, 0, True)
End Reverse


In [22]:
from checkpoint_schedules import TwoLevelCheckpointSchedule
revolver = TwoLevelCheckpointSchedule(2, 10)
chk_manager.execute(revolver)


*
|--->--->                  Forward(0, 2, True, False, 'disk')
       *
       |--->--->            Forward(2, 4, True, False, 'disk')
End Forward
       +                        Copy(2, 'disk', False)
       |--->               Forward(2, 3, False, False, 'NONE')
           |--->            Forward(3, 4, False, True, 'RAM')
           <---|            Reverse(4, 3, True)
       +                        Copy(2, 'disk', False)
       |--->               Forward(2, 3, False, True, 'RAM')
       <---|               Reverse(3, 2, True)
+                        Copy(0, 'disk', False)
|--->                     Forward(0, 1, False, False, 'NONE')
   |--->                  Forward(1, 2, False, True, 'RAM')
   <---|                  Reverse(2, 1, True)
+                        Copy(0, 'disk', False)
|--->                     Forward(0, 1, False, True, 'RAM')
<---|                     Reverse(1, 0, True)
End Reverse


In [23]:
from checkpoint_schedules import PeriodicDiskCheckpointSchedule
revolver = PeriodicDiskCheckpointSchedule(2)
chk_manager.execute(revolver)

*
|--->--->                  Forward(0, 2, True, False, 'disk')
       *
       |--->--->            Forward(2, 4, True, False, 'disk')
End Forward
       +                        Copy(2, 'disk', False)
       |--->--->            Forward(2, 4, False, True, 'RAM')
       <---<---|            Reverse(4, 2, True)
+                        Copy(0, 'disk', False)
|--->--->                  Forward(0, 2, False, True, 'RAM')
<---<---|                  Reverse(2, 0, True)
End Reverse


*
|--->--->--->               Forward(0, 3, True, False, 'RAM')
           |--->            Forward(3, 4, False, True, 'RAM')
End Forward
           <---|            Reverse(4, 3, True)
+                        Copy(0, 'RAM', False)
|--->--->                  Forward(0, 2, False, False, 'NONE')
       |--->               Forward(2, 3, False, True, 'RAM')
       <---|               Reverse(3, 2, True)
+                        Copy(0, 'RAM', False)
|--->                     Forward(0, 1, False, False, 'NONE')
   |--->                  Forward(1, 2, False, True, 'RAM')
   <---|                  Reverse(2, 1, True)
+                        Copy(0, 'RAM', True)
|--->                     Forward(0, 1, False, True, 'RAM')
<---|                     Reverse(1, 0, True)
End Reverse


This first example gives the basics of executions involving an adjoint-based gradient using *checkpoint_schedules* package. The next section shows an example an application of adjoint-based gradient problem.